In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vo
import model_validation.ciff_sam_results as csr

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/nathaniel/scratch__gitignore__
ndbs
Fri Aug 27 14:53:23 PDT 2021


In [2]:
%load_ext autoreload
%autoreload 2

In [15]:
import model_validation.ciff_sam_results as csr

# Test models dataframe for use in `get_count_data_path` function below

In [16]:
models = pd.DataFrame(
    [
        [2.4, 'v2.4_corrected_fertility', '2021_08_03_15_08_32'],
        [2.5, 'v2.5_stunting', '2021_08_05_16_17_12'],
        [3.0, 'v3.0_sq_lns', '2021_08_16_17_54_19'],
        [3.1, 'v3.1_sq_lns_stunting_stratified', '2021_08_24_10_28_32']
    ],
    columns=['model_id', 'model_name', 'run_id']
)
models

,model_id,model_name,run_id
0,2.4,v2.4_corrected_fertility,2021_08_03_15_08_32
1,2.5,v2.5_stunting,2021_08_05_16_17_12
2,3.0,v3.0_sq_lns,2021_08_16_17_54_19
3,3.1,v3.1_sq_lns_stunting_stratified,2021_08_24_10_28_32


In [26]:
models.loc[models.model_id==2.5, 'run_id']

1    2021_08_05_16_17_12
Name: run_id, dtype: object

In [25]:
models.set_index('model_id').loc[2.5, 'run_id']

'2021_08_05_16_17_12'

In [39]:
models.set_index('model_id').loc[2.5]

model_name          v2.5_stunting
run_id        2021_08_05_16_17_12
Name: 2.5, dtype: object

In [41]:
models.set_index('model_id').loc[2.5, ['model_name', 'run_id']]

model_name          v2.5_stunting
run_id        2021_08_05_16_17_12
Name: 2.5, dtype: object

In [42]:
model_name, run_id = models.set_index('model_id').loc[2.5, ['model_name', 'run_id']]
print(model_name, run_id)

v2.5_stunting 2021_08_05_16_17_12


In [55]:
model_metadata = models.set_index('model_id').loc[2.5] 
model_metadata.loc[['model_name', 'run_id']]
# model_name, run_id = 
# print(model_name, run_id)

model_name          v2.5_stunting
run_id        2021_08_05_16_17_12
Name: 2.5, dtype: object

In [79]:
model_metadata

model_name          v2.5_stunting
run_id        2021_08_05_16_17_12
Name: 2.5, dtype: object

In [33]:
fake_models = models.append(
    pd.Series([2.5, 'v2.5_stunting', '2021_08_07_14_10_05'], index=models.columns),
    ignore_index=True
)
fake_models

,model_id,model_name,run_id
0,2.4,v2.4_corrected_fertility,2021_08_03_15_08_32
1,2.5,v2.5_stunting,2021_08_05_16_17_12
2,3.0,v3.0_sq_lns,2021_08_16_17_54_19
3,3.1,v3.1_sq_lns_stunting_stratified,2021_08_24_10_28_32
4,2.5,v2.5_stunting,2021_08_07_14_10_05


In [36]:
fake_models.set_index('model_id').loc[2.5, 'run_id']

model_id
2.5    2021_08_05_16_17_12
2.5    2021_08_07_14_10_05
Name: run_id, dtype: object

In [38]:
fake_models.set_index('model_id').loc[2.5]

,model_name,run_id
model_id,,
2.5,v2.5_stunting,2021_08_05_16_17_12
2.5,v2.5_stunting,2021_08_07_14_10_05


In [40]:
fake_models.set_index('model_id').loc[2.5, ['model_name', 'run_id']]

,model_name,run_id
model_id,,
2.5,v2.5_stunting,2021_08_05_16_17_12
2.5,v2.5_stunting,2021_08_07_14_10_05


In [43]:
model_name, run_id = fake_models.set_index('model_id').loc[2.5, ['model_name', 'run_id']]
print(model_name, run_id)

model_name run_id


In [56]:
fake_model_metadata = fake_models.set_index('model_id').loc[2.5] 
fake_model_metadata.loc[['model_name', 'run_id']]
# model_name, run_id = 

KeyError: "None of [Index(['model_name', 'run_id'], dtype='object', name='model_id')] are in the [index]"

In [80]:
model_metadata

model_name          v2.5_stunting
run_id        2021_08_05_16_17_12
Name: 2.5, dtype: object

In [97]:
model_metadata['run_id']=='2021_08_05_16_17_12'

True

In [99]:
type(model_metadata['run_id']=='2021_08_05_16_17_12') is bool

True

In [81]:
fake_model_metadata

,model_name,run_id
model_id,,
2.5,v2.5_stunting,2021_08_05_16_17_12
2.5,v2.5_stunting,2021_08_07_14_10_05


In [82]:
model_metadata['model_name']

'v2.5_stunting'

In [83]:
fake_model_metadata['model_name']

model_id
2.5    v2.5_stunting
2.5    v2.5_stunting
Name: model_name, dtype: object

In [95]:
fake_model_metadata['run_id'] == '2021_08_05_16_17_12'

model_id
2.5     True
2.5    False
Name: run_id, dtype: bool

In [110]:
fake_model_metadata = fake_models.set_index('model_id').loc[2.5] 
fake_model_metadata.loc[fake_model_metadata.run_id=='2021_08_07_14_10_05'].squeeze()

model_name          v2.5_stunting
run_id        2021_08_07_14_10_05
Name: 2.5, dtype: object

In [68]:
fake_model_metadata = fake_models.set_index('model_id').loc[2.5] 
fake_model_metadata.loc[fake_model_metadata.run_id=='2021_08_07_14_10_05', 'model_name'].iat[0]

'v2.5_stunting'

In [125]:
model_metadata

model_name          v2.5_stunting
run_id        2021_08_05_16_17_12
Name: 2.5, dtype: object

In [170]:
model_metadata.query("run_id=='2021_08_05_16_17_12'")

AttributeError: 'Series' object has no attribute 'query'

In [126]:
fake_model_metadata

,model_name,run_id
model_id,,
2.5,v2.5_stunting,2021_08_05_16_17_12
2.5,v2.5_stunting,2021_08_07_14_10_05


In [127]:
model_metadata[['run_id']]

run_id    2021_08_05_16_17_12
Name: 2.5, dtype: object

In [128]:
fake_model_metadata[['run_id']]

,run_id
model_id,
2.5,2021_08_05_16_17_12
2.5,2021_08_07_14_10_05


In [129]:
fake_model_metadata['run_id']

model_id
2.5    2021_08_05_16_17_12
2.5    2021_08_07_14_10_05
Name: run_id, dtype: object

# Write and test function to to get the results directory for a specified model number

In [207]:
project_results_dir = '/ihme/costeffectiveness/results/vivarium_ciff_sam'

def get_count_data_path1(model_id, run_id=None, models_df=None):
    if models_df is None:
        models_df = models
    model_metadata = models_df.set_index('model_id').loc[model_id]
    if run_id is None:
        if len(model_metadata[['run_id']]) > 1:
            raise ValueError(
                f"You must specify the run_id for model_id {model_id} because"
                f" there is more than one run_id:\n{model_metadata['run_id']}"
            )
        model_name, run_id = model_metadata.loc[['model_name', 'run_id']]
    else:
        matching_run = model_metadata['run_id']==run_id
        if type(matching_run) is bool:
            # There was only one row corresponding to this model, and model_metadata is a Series
            if not matching_run:
                raise ValueError(
                    f"No run_id {run_id} for model_id {model_id}:\n"
                    f"{model_metadata}"
                )
        else:
            if len(model_metadata.loc[matching_run]) == 0:
                raise ValueError(
                    f"No run_id {run_id} for model_id {model_id}:\n"
                    f"{model_metadata}"
                )
            else:
                # There was more than one row, model_metadata is a DataFrame, and type(matching_run) is Series
                model_metadata = model_metadata.loc[matching_run].squeeze()
        model_name = model_metadata['model_name']
        print(model_metadata, len(model_metadata), model_name,'\n',run_id)
    model_count_data_path = f'{project_results_dir}/{model_name}/ciff_sam/{run_id}/count_data/'
    return model_count_data_path

def get_count_data_path(model_id, run_id=None, models_df=models):
    model_metadata = models_df.set_index('model_id').loc[model_id]
    if run_id is None:
        if len(model_metadata[['run_id']]) > 1:
            raise ValueError(
                f"You must specify the run_id for model_id {model_id} because"
                f" there is more than one run_id:\n{model_metadata}"
            )
        model_name, run_id = model_metadata.loc[['model_name', 'run_id']]
    else:
        matching_run = model_metadata['run_id']==run_id
        if isinstance(matching_run, bool) and matching_run:
            # There was only one row corresponding to this model, and model_metadata is a Series
            pass
        elif isinstance(matching_run, pd.Series) and len(model_metadata.loc[matching_run]) > 0:
            # There was more than one row, so model_metadata is a DataFrame and matching_run is a Series
            # Filter to metadata for the requested run_id and convert to a Series:
                model_metadata = model_metadata.loc[matching_run].squeeze()
        else:
            raise ValueError(
                    f"No run_id {run_id} for model_id {model_id}:\n"
                    f"{model_metadata}"
                )
        model_name = model_metadata['model_name']
#         print(model_metadata, len(model_metadata), model_name,'\n',run_id)
    model_count_data_path = f'{project_results_dir}/{model_name}/ciff_sam/{run_id}/count_data/'
    return model_count_data_path

print(get_count_data_path(2.5) == get_count_data_path(2.5, '2021_08_05_16_17_12'))
get_count_data_path(2.5)

True


'/ihme/costeffectiveness/results/vivarium_ciff_sam/v2.5_stunting/ciff_sam/2021_08_05_16_17_12/count_data/'

In [211]:
csr.get_count_data_path(2.5, None, fake_models)

ValueError: You must specify the run_id for model_id 2.5 because there is more than one run_id:
             model_name               run_id
model_id                                    
2.5       v2.5_stunting  2021_08_05_16_17_12
2.5       v2.5_stunting  2021_08_07_14_10_05

In [212]:
csr.get_count_data_path(2.5, '2021_08_05_16_17_12', fake_models)

'/ihme/costeffectiveness/results/vivarium_ciff_sam/v2.5_stunting/ciff_sam/2021_08_05_16_17_12/count_data/'

In [213]:
csr.get_count_data_path(2.5, '2021_08_05_16_17_12', fake_models) == get_count_data_path(2.5)

True

In [214]:
csr.get_count_data_path(2.5, '45559')

ValueError: No run_id 45559 for model_id 2.5:
model_name          v2.5_stunting
run_id        2021_08_05_16_17_12
Name: 2.5, dtype: object

In [215]:
csr.get_count_data_path(2.5, '45559', fake_models)

ValueError: No run_id 45559 for model_id 2.5:
             model_name               run_id
model_id                                    
2.5       v2.5_stunting  2021_08_05_16_17_12
2.5       v2.5_stunting  2021_08_07_14_10_05

In [216]:
csr.get_count_data_path(2.5) == csr.get_count_data_path(2.5, '2021_08_05_16_17_12')

True

In [200]:
isinstance(True, bool)

True

# Test VivariumMeasures class

In [76]:
path25 = get_count_data_path(2.5)
path25

'/ihme/costeffectiveness/results/vivarium_ciff_sam/v2.5_stunting/ciff_sam/2021_08_05_16_17_12/count_data/'

In [77]:
data = csr.VivariumMeasures.from_directory(path25)
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [78]:
type(data)

model_validation.ciff_sam_results.VivariumMeasures

In [114]:
orig_data = vto.VivariumTransformedOutput.from_directory(path25)
orig_data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [115]:
data2 = csr.VivariumMeasures(orig_data)
data2.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

## Test mixin method `__eq__`...

It doesn't work because it uses `==` for dataframes instead of `.equals()`

In [116]:
data2 == data

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Test classmethod `from_model_spec`

In [217]:
data24 = csr.VivariumMeasures.from_model_spec(2.4)
data24.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [218]:
data24.deaths

,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,female,2022,diarrheal_diseases,death,29,baseline,22.0,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,death,29,baseline,6.0,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,death,29,baseline,3.0,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,death,29,baseline,0.0,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,susceptible_to_child_wasting,2_to_4
17277,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,mild_child_wasting,2_to_4
17278,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,moderate_acute_malnutrition,2_to_4
17279,male,2026,severe_acute_malnutrition,death,946,baseline,4.0,severe_acute_malnutrition,2_to_4


In [219]:
type(data24)

model_validation.ciff_sam_results.VivariumMeasures

In [221]:
data30 = csr.VivariumMeasures.from_model_spec(3.0)
data30.table_names()

['stunting_state_person_time']

In [222]:
data30.stunting_state_person_time

,cause,measure,input_draw,scenario,value,sq_lns
0,,state_person_time,29,baseline,0.000000e+00,covered
1,,state_person_time,29,baseline,1.204420e+06,uncovered
2,,state_person_time,29,baseline,0.000000e+00,covered
3,,state_person_time,29,baseline,5.181539e+05,uncovered
...,...,...,...,...,...,...
188,,state_person_time,946,treatment_and_prevention,3.078697e+05,covered
189,,state_person_time,946,treatment_and_prevention,1.515505e+05,uncovered
190,,state_person_time,946,treatment_and_prevention,2.579365e+05,covered
191,,state_person_time,946,treatment_and_prevention,1.271639e+05,uncovered


# Test `clean_transformed_data` function

In [226]:
[x for x in data24 if x.endswith('transition_count')]

['wasting_transition_count', 'disease_transition_count']

In [227]:
data.wasting_transition_count

,sex,year,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,1245.0,early_neonatal
1,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,3803.0,late_neonatal
2,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,29178.0,1-5_months
3,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,34038.0,6-11_months
...,...,...,...,...,...,...,...
5036,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,baseline,3330.0,1-5_months
5037,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,baseline,4391.0,6-11_months
5038,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,baseline,8691.0,12_to_23_months
5039,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,baseline,23097.0,2_to_4


In [231]:
clean_data = csr.clean_transformed_data(data)
clean_data.wasting_transition_count

,sex,year,measure,input_draw,scenario,value,age,transition
0,female,2022,transition_count,29,baseline,1245.0,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,3803.0,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,29178.0,1-5_months,mild_child_wasting_to_moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,34038.0,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...
5036,male,2026,transition_count,946,baseline,3330.0,1-5_months,susceptible_to_child_wasting_to_mild_child_was...
5037,male,2026,transition_count,946,baseline,4391.0,6-11_months,susceptible_to_child_wasting_to_mild_child_was...
5038,male,2026,transition_count,946,baseline,8691.0,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...
5039,male,2026,transition_count,946,baseline,23097.0,2_to_4,susceptible_to_child_wasting_to_mild_child_was...


In [232]:
clean_data.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,414.090349,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1234.535250,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,8926.954141,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,10434.387406,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,30258.041068,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,35695.241615,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,70879.949350,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,215444.187543,2_to_4


In [230]:
clean_data.disease_state_person_time

,sex,year,cause_state,measure,input_draw,scenario,value,stunting_state,age,cause
0,female,2022,diarrheal_diseases,state_person_time,29,baseline,43.489391,cat4,early_neonatal,diarrheal_diseases
1,female,2022,diarrheal_diseases,state_person_time,29,baseline,0.000000,cat3,early_neonatal,diarrheal_diseases
2,female,2022,diarrheal_diseases,state_person_time,29,baseline,0.000000,cat2,early_neonatal,diarrheal_diseases
3,female,2022,diarrheal_diseases,state_person_time,29,baseline,0.000000,cat1,early_neonatal,diarrheal_diseases
...,...,...,...,...,...,...,...,...,...,...
17276,male,2026,susceptible_to_measles,state_person_time,946,baseline,121118.023272,cat4,2_to_4,measles
17277,male,2026,susceptible_to_measles,state_person_time,946,baseline,67325.987680,cat3,2_to_4,measles
17278,male,2026,susceptible_to_measles,state_person_time,946,baseline,66608.265572,cat2,2_to_4,measles
17279,male,2026,susceptible_to_measles,state_person_time,946,baseline,63010.346338,cat1,2_to_4,measles


In [242]:
csr.clean_transformed_data(data30).stunting_state_person_time

,cause,measure,input_draw,scenario,value,sq_lns
0,,state_person_time,29,baseline,0.000000e+00,covered
1,,state_person_time,29,baseline,1.204420e+06,uncovered
2,,state_person_time,29,baseline,0.000000e+00,covered
3,,state_person_time,29,baseline,5.181539e+05,uncovered
...,...,...,...,...,...,...
188,,state_person_time,946,treatment_and_prevention,3.078697e+05,covered
189,,state_person_time,946,treatment_and_prevention,1.515505e+05,uncovered
190,,state_person_time,946,treatment_and_prevention,2.579365e+05,covered
191,,state_person_time,946,treatment_and_prevention,1.271639e+05,uncovered


# Test `cleaned_from_model_spec` class method

In [246]:
clean_data25 = csr.VivariumMeasures.cleaned_from_model_spec(2.5)
clean_data25.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [247]:
clean_data25.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,414.090349,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1234.535250,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,8926.954141,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,10434.387406,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,30258.041068,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,35695.241615,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,70879.949350,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,215444.187543,2_to_4


In [245]:
clean_data25.disease_transition_count

,sex,year,measure,input_draw,scenario,value,stunting_state,age,transition
0,female,2022,transition_count,29,baseline,2475.0,cat4,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...
1,female,2022,transition_count,29,baseline,0.0,cat3,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...
2,female,2022,transition_count,29,baseline,0.0,cat2,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...
3,female,2022,transition_count,29,baseline,0.0,cat1,early_neonatal,diarrheal_diseases_to_susceptible_to_diarrheal...
...,...,...,...,...,...,...,...,...,...
17276,male,2026,transition_count,946,baseline,447.0,cat4,2_to_4,susceptible_to_measles_to_measles
17277,male,2026,transition_count,946,baseline,243.0,cat3,2_to_4,susceptible_to_measles_to_measles
17278,male,2026,transition_count,946,baseline,368.0,cat2,2_to_4,susceptible_to_measles_to_measles
17279,male,2026,transition_count,946,baseline,642.0,cat1,2_to_4,susceptible_to_measles_to_measles
